In [2]:
from global_config import config
import pandas as pd
import numpy as np
import rpy2
import os


data_dir_mnps = config.get_property('geo_dir')
data_dir      = config.get_property('data_dir')
fb_ppl_data   = config.get_property('covid_fb')
results_dir   = config.get_property('results_dir')

In [6]:
rt_df = pd.read_csv( os.path.join(results_dir, 'bog_rt', 'rt_df_bog_confirmation.csv') )
#rt_df = rt_df[rt_df["variable"]=='R']
rt_df["variable"].unique()

array(['R', 'day_of_week', 'delay_mean', 'delay_sd', 'growth_rate',
       'gt_mean', 'gt_sd', 'infections', 'prior_infections',
       'reported_cases', 'reporting_overdispersion'], dtype=object)